In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
data = pd.read_csv('titanic_train.csv')

In [19]:
# fixing data

def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age
    

data['Age'] = data[['Age','Pclass']].apply(impute_age,axis=1)

data.drop('Cabin',axis=1,inplace=True)
data.dropna(inplace=True)

sex = pd.get_dummies(data['Sex'],drop_first=True)
embark = pd.get_dummies(data['Embarked'],drop_first=True)
data.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [20]:
from sklearn.model_selection import train_test_split
Y = pd.DataFrame(data['Survived'], columns=["Survived"])
X_train, X_test, Y_train, Y_test = train_test_split(data.drop('Survived',axis=1), Y, test_size=0.20, random_state=101)
X_train.insert(0, 'theta0', 1)
X_test.insert(0, 'theta0', 1)


In [21]:
"""
logistic regression
----------------------------------------------------------
Return the gradient of the J function at the current theta
g(theta) = (predicted-real) * x / m
"""
def g(x, y, theta):
    m = len(x.index)
    diff = h(theta, x) - y
    return np.dot(x.transpose(), diff) / m

def h(theta, x):
    #add sigmoidal function
    return sig(np.dot(x, theta))

def sig(x):
    return 1/(1 + np.exp(-1 * x))

def gradient_descent(x, y, theta, alpha = 0.1, n_iter = 50):
    for iteration in range(n_iter):
        theta = theta - alpha * g(x, y, theta)
    return theta

def estimate(theta, x):
    yhat = h(theta, x)
    yhat[yhat >= 0.5] = 1
    yhat[yhat < 0.5] = 0
    return yhat


In [22]:
theta_0 = pd.DataFrame(0, index=range(7), columns=["theta"])

theta_final = gradient_descent(X_train, Y_train, theta_0, alpha = 0.1, n_iter = 100)

C:\Users\julie\AppData\Local\Temp\ipykernel_28848\349693804.py:17: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-1 * x))


In [23]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

predictions = estimate(theta_final, X_train)
#print(predictions)

print(classification_report(Y_train, predictions))
print("Training Accuracy:", accuracy_score(Y_train, predictions))

predictions = estimate(theta_final, X_test)
print(classification_report(Y_test,predictions))
print("Testing Accuracy:", accuracy_score(Y_test, predictions))


              precision    recall  f1-score   support

           0       0.63      0.99      0.77       442
           1       0.67      0.02      0.04       269

    accuracy                           0.63       711
   macro avg       0.65      0.51      0.41       711
weighted avg       0.64      0.63      0.49       711

Training Accuracy: 0.6258790436005626
              precision    recall  f1-score   support

           0       0.60      0.97      0.74       107
           1       0.40      0.03      0.05        71

    accuracy                           0.60       178
   macro avg       0.50      0.50      0.40       178
weighted avg       0.52      0.60      0.47       178

Testing Accuracy: 0.5955056179775281


C:\Users\julie\AppData\Local\Temp\ipykernel_28848\349693804.py:17: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-1 * x))
C:\Users\julie\AppData\Local\Temp\ipykernel_28848\349693804.py:17: RuntimeWarning: overflow encountered in exp
  return 1/(1 + np.exp(-1 * x))


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

X_train, X_test, Y_train, Y_test = train_test_split(data.drop('Survived',axis=1), Y, test_size=0.20, random_state=101)

logmodel = LogisticRegression()
logmodel.fit(X_train,Y_train)

predictions = logmodel.predict(X_train)
print(classification_report(Y_train,predictions))
print("Training Accuracy:", accuracy_score(Y_train, predictions))

predictions = logmodel.predict(X_test)
print(classification_report(Y_test,predictions))
print("Testing Accuracy:", accuracy_score(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.71      0.87      0.78       442
           1       0.65      0.41      0.50       269

    accuracy                           0.69       711
   macro avg       0.68      0.64      0.64       711
weighted avg       0.68      0.69      0.67       711

Training Accuracy: 0.6919831223628692
              precision    recall  f1-score   support

           0       0.75      0.88      0.81       107
           1       0.75      0.56      0.65        71

    accuracy                           0.75       178
   macro avg       0.75      0.72      0.73       178
weighted avg       0.75      0.75      0.74       178

Testing Accuracy: 0.7528089887640449


c:\Progs\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
